In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, pipeline, metrics
from sklearn.model_selection import GridSearchCV

In [2]:
# Define Metric (Quadratic Weighted Kappa)
# The following 3 functions are taken from https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [3]:
train_clean = pd.read_csv("./train_clean.csv")
train_clean = train_clean.fillna("")
train_clean.head()

,query,product_title,product_description,median_relevance,relevance_variance
0,bridal shower decoration,accent pillow heart design red black,red satin accent pillow embroider heart black ...,1,0.000
1,lead christmas light,set battery operate multi lead train christmas...,set battery operate train christmas light item...,4,0.000
2,projector,viewsonic pro dlp multimedia projector,,4,0.471
3,wine rack,concept housewares wr solid wood ceiling wall ...,like silent sturdy tree southern enterprise bi...,4,0.000
4,light bulb,wintergreen light christmas lead light bulb pack,wtgr feature nickel base average hour acrylic ...,2,0.471


For each observation, combine **query**, **product_title** and **product_description** to create a giant text. By doing so we can capture the similiaritis between these columns. For example, if a input query is "lead christmas light", and both product_title and product_description contain the key words "lead", "christmas" and "light", then the vectorized text will be longer in these three dimensions and shorter in others. If the input query share no common word with product_title or product_description, the vectorized text will have the same length in all the dimensions. <br/>
<br/>
However, there is one potential problem in this method. If the input query is totally different from product_title and product_description, but product_title and product_description are similar, we will still get a feature vector seeming to have high relevance. So to avoid such issue, it would be better to combine one column with the input query. Since there are much more missing values in product_description, I choose to combine "query" and "product_title".

In [4]:
tmp = train_clean["query"] + [" "] + train_clean["product_title"]
traindata = tmp.values.tolist()

In [5]:
# create labels
y = train_clean["median_relevance"].values

In [7]:
test_clean = pd.read_csv("./test_clean.csv")
test_clean = test_clean.fillna("")
tmp = test_clean["query"] + [" "] + test_clean["product_title"]
testdata = tmp.values.tolist()

### Count Vectorizer

In [8]:
ctv = CountVectorizer(analyzer = "word", \
                      tokenizer = None, \
                      preprocessor = None, \
                      stop_words = None, \
                      max_features = None) 

In [9]:
ctv.fit(traindata)
X =  ctv.transform(traindata) 
X_test = ctv.transform(testdata)

In [10]:
# performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). 
svd = TruncatedSVD()
# Standardize
scl = StandardScaler()
# Use SVM
svm_model = SVC()

clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('svm', svm_model)])
# Tuning hyper-parameter with grid search
param_grid = {'svd__n_components' : [200, 400],
              'svm__C': [10, 12]}

In [12]:
kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better=True)

model = GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                     verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)
model.fit(X, y)
print("Best score: %0.4f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Get best model
best_model = model.best_estimator_

# Fit model with best parameters optimized for quadratic_weighted_kappa
best_model.fit(X,y)
preds = best_model.predict(X_test)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] svd__n_components=200, svm__C=10 ................................
[CV] svd__n_components=200, svm__C=10 ................................
[CV] svd__n_components=200, svm__C=12 ................................
[CV] svd__n_components=200, svm__C=12 ................................
[CV] svd__n_components=400, svm__C=10 ................................
[CV] svd__n_components=400, svm__C=10 ................................
[CV] svd__n_components=400, svm__C=12 ................................
[CV] svd__n_components=400, svm__C=12 ................................
[CV]  svd__n_components=200, svm__C=10, score=0.582924458254033, total=  40.7s
[CV]  svd__n_components=200, svm__C=12, score=0.5848008153947173, total=  40.8s


[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   52.0s remaining:  2.6min


[CV]  svd__n_components=200, svm__C=10, score=0.5483821209704955, total=  41.6s
[CV]  svd__n_components=200, svm__C=12, score=0.550293458448539, total=  41.6s


[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:   53.0s remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:   53.0s remaining:   53.0s


[CV]  svd__n_components=400, svm__C=10, score=0.5340066732457942, total= 1.3min


...

[CV]  svd__n_components=400, svm__C=12, score=0.5407554744487292, total= 1.3min


[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:  1.5min remaining:   30.5s


[CV]  svd__n_components=400, svm__C=10, score=0.546330497938208, total= 1.3min
[CV]  svd__n_components=400, svm__C=12, score=0.5350237903158717, total= 1.3min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  1.6min finished


Best score: 0.5675
Best parameters set:
	svd__n_components: 200
	svm__C: 12


In [14]:
# Create submission file
test = pd.read_csv("./data/test.csv")
idx = test["id"].values
submission = pd.DataFrame({"id": idx, "prediction": preds})
submission.to_csv("CTV_SVM.csv", index=False)

Private leader board score 0.60345.

## TFIDF

In [15]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
        strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
        ngram_range=(1, 5), use_idf=1,smooth_idf=1,sublinear_tf=1,
        stop_words = 'english')

tfv.fit(traindata)
X =  tfv.transform(traindata) 
X_test = tfv.transform(testdata)

In [16]:
# performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). 
svd = TruncatedSVD()
# Standardize
scl = StandardScaler()
# Use SVM
svm_model = SVC()

clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('svm', svm_model)])
# Tuning hyper-parameter with grid search
param_grid = {'svd__n_components' : [200, 400],
              'svm__C': [10, 12]}

In [17]:
# Kappa Scorer 
kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)

# Initialize Grid Search Model
model = GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                     verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(X, y)
print("Best score: %0.4f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Get best model
best_model = model.best_estimator_

# Fit model with best parameters optimized for quadratic_weighted_kappa
best_model.fit(X,y)
preds = best_model.predict(X_test)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] svd__n_components=200, svm__C=10 ................................
[CV] svd__n_components=200, svm__C=10 ................................
[CV] svd__n_components=200, svm__C=12 ................................
[CV] svd__n_components=200, svm__C=12 ................................
[CV] svd__n_components=400, svm__C=10 ................................
[CV] svd__n_components=400, svm__C=10 ................................
[CV] svd__n_components=400, svm__C=12 ................................
[CV] svd__n_components=400, svm__C=12 ................................
[CV]  svd__n_components=200, svm__C=10, score=0.5132496473226699, total=  44.1s
[CV]  svd__n_components=200, svm__C=12, score=0.522284488387325, total=  44.2s


[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   53.5s remaining:  2.7min


[CV]  svd__n_components=200, svm__C=10, score=0.5177033785906953, total=  44.9s
[CV]  svd__n_components=200, svm__C=12, score=0.5238631720572573, total=  45.1s


[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:   54.1s remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:   54.3s remaining:   54.3s


[CV]  svd__n_components=400, svm__C=10, score=0.5349037749231415, total= 1.1min


...

[CV]  svd__n_components=400, svm__C=10, score=0.5624675916806688, total= 1.1min


[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:  1.3min remaining:   26.6s


[CV]  svd__n_components=400, svm__C=12, score=0.5407940266777758, total= 1.2min
[CV]  svd__n_components=400, svm__C=12, score=0.5664372746028488, total= 1.2min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  1.4min finished


Best score: 0.5536
Best parameters set:
	svd__n_components: 400
	svm__C: 12


In [18]:
# Create submission file
submission = pd.DataFrame({"id": idx, "prediction": preds})
submission.to_csv("TFIDF_SVM.csv", index=False)

Private leader board score 0.59078.